In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Modelos

In [2]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 739.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successf

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudar você hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-6384bc4d-b11c-47c6-abc5-490a84848775-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


## Prompts

### Templates Simples

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [6]:
response = llm.invoke(prompt)

print(response.content)

A inteligência artificial é o futuro!


### Templates de Mensagens

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-1da0c478-4b1c-4321-bcca-edb82fccf946-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [8]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [9]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hello, how are you?')]


In [10]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [12]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [13]:
chain = prompt_template | llm | parser

In [14]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [15]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [16]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [20]:
prompt_template = ChatPromptTemplate.from_template('Faça uma piada sobre {topico}')

chain = prompt_template | llm | parser

chain.invoke({'topico': 'matemática'})

'Por que o livro de matemática se suicidou?\n\nPorque tinha muitos problemas! 😄'

### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [22]:
prompt_template = ChatPromptTemplate.from_template('Classifique em sentimento positivo, neutro ou negativo o seguinte texto: {texto}')

chain = prompt_template | llm | parser

chain.invoke({'texto': 'A comida daquele restaurante é sensacional.'})

'O sentimento do texto é positivo.'

### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [26]:
prompt_template = ChatPromptTemplate.from_messages(
    [
      ("system", "Você é um programador especialista em python e converte descrições de funções em código python. Converta em código python as descrições recebidas."),
      ("user", "{texto}"),
      ]

)

chain = prompt_template | llm | parser

resultado = chain.invoke({"texto": "escreva o código python de uma função que retorne a média entre dois números"})

print(resultado)

Aqui está o código Python para uma função que retorna a média entre dois números:

```python
def media_dois_numeros(num1, num2):
    return (num1 + num2) / 2
```

Você pode usar essa função passando dois números como argumentos para obter a média. Por exemplo:

```python
resultado = media_dois_numeros(10, 20)
print(resultado)  # Saída: 15.0
```


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [27]:
prompt_template = ChatPromptTemplate.from_template('Explique de forma simplificada e depois traduza sua explicação do seguinte tópicp: {topico}')

chain = prompt_template | llm | parser

chain.invoke({'topico': 'Exploração espacial'})

'**Explicação Simplificada:**\n\nA exploração espacial é o processo de investigar o espaço além da Terra. Isso é feito por meio de naves espaciais, sondas, telescópios e outros equipamentos que ajudam os cientistas a entender mais sobre o universo, planetas, estrelas e galáxias. A exploração pode incluir missões tripuladas, como as que levam astronautas à Lua ou à Estação Espacial Internacional, e missões não tripuladas, que enviam robôs e sondas para estudar outros planetas, como Marte ou Júpiter. O objetivo é descobrir mais sobre como o universo funciona, a origem da vida e se existe vida em outros lugares.\n\n**Tradução:**\n\n**Simplified Explanation:**\n\nSpace exploration is the process of investigating space beyond Earth. This is done through spacecraft, probes, telescopes, and other equipment that help scientists learn more about the universe, planets, stars, and galaxies. Exploration can include crewed missions, like those that take astronauts to the Moon or the International S

### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [28]:
prompt_template = ChatPromptTemplate.from_messages(
    [
      ("system", "Você é um funcionário responsável por fornecer informações sobre o Cesar School. Responda as perguntas a respeito da Cesar School recebidas."),
      ("user", "{texto}"),
      ]

)

chain = prompt_template | llm | parser

resultado = chain.invoke({"texto": "Quais são os cursos disponíveis?"})

print(resultado)

A Cesar School oferece uma variedade de cursos nas áreas de tecnologia e inovação. Alguns dos cursos disponíveis incluem:

1. **MBA em Gestão de Produtos** - Focado em desenvolvimento e gestão de produtos digitais.
2. **MBA em Gestão de Tecnologia da Informação** - Voltado para a administração e gestão de TI nas organizações.
3. **Cursos de Formação em Desenvolvimento de Software** - Incluindo programação, desenvolvimento web e mobile.
4. **Cursos de Design** - Como Design de Interação e Design de Experiência do Usuário (UX).
5. **Data Science e Analytics** - Cursos que abordam análise de dados e técnicas de ciência de dados.
6. **Cursos de Empreendedorismo e Inovação** - Focados em capacitar empreendedores e fomentar a inovação.

Para informações mais detalhadas sobre cada curso, incluindo datas e pré-requisitos, recomendo visitar o site oficial da Cesar School.
